In [2]:
import warnings
warnings.filterwarnings("ignore")
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
device = "cuda"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
import PyPDF2
segments = []
def read_pdf_segments(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            segment = page.extract_text()
            segments.append(segment)
        
        return segments
read_pdf_segments("godan.pdf")

[' \n1 www.hindustanbooks.com  \n गोदान  \nप्रेमचंद  \n  ',
 " \n2 www.hindustanbooks.com  \n भाग 1 \nहोरीराम ने दोनों बैलों को सानी -पानी दे कर अपनी स्त्री धननया से कहा - गोबर को ऊख गोड़ने भेज देना। मैं न \nजाने कब लौट ूँ। जरा मेरी लाठी दे दे। धननया के दोनों हाथ गोबर से भरे थे। उपले पाथ कर आई थी। बोली - \nअरे, कुछ रस -पानी तो कर लो। ऐसी जल्दी क्या है ? होरी ने अपने झुर्ररयों से भरे हुए माथे को नसकोड़ कर कहा \n- तुझे रस -पानी की पड़ी है , मुझे यह चचता है कक अबेर हो गई तो मानलक से भेंट न होगी। असनान -प जा करने \nलगेंगे, तो घंटों बैठे बीत जायगा। 'इसी से तो कहती हूँ , कुछ जलपान कर लो और आज न जाओगे तो  कौन हरज \nहोगा! अभी तो परसों गए थे। ' \n'त  जो बात नहीं समझती , उसमें टाूँग क्यों अड़ाती है भाई! मेरी लाठी दे दे और अपना काम देख। यह इसी \nनमलते -जुलते रहने का परसाद है कक अब तक जान बची हुई है , नहीं कहीं पता न लगता कक ककधर गए। गाूँव में \nइतने आदमी तो हैं , ककस पर बेदखली नहीं आई , ककस पर कुड़की नहीं आई। जब द सरे के पाूँवों -तले अपनी गददन \nदबी हुई है , तो उन पाूँवों को सहलाने में ही कुसल है

In [ ]:
# books = {
#         "book1": "KarmaYoga.pdf",
#         "book2": "PracticalVedanta.pdf",
#         "book3": "godan.pdf",
#         "book4": "abridged-autobiography-hindi.pdf",
#         "book5": "narayan-kwach.pdf"
# }
# segmented_books = {}
# for book in books:
#     segmented_books[book] = read_pdf_segments(books[book])
# segmented_books.keys()

In [14]:
res = ""
resfin = []
question = "how did heera drown in debt"
for segment in segments:
  messages = [
    {
      "role": "user", "content":'''I will provide you with a question, and some text.
       Your job is to tell me if the question that I provided is relevant to the given text or is incoherent. Also if you find it relevant can u give the part of the text where you find its resemblance''',
    },
    {
      "role": "assistant", "content":"Sure, please provide me with the text and the question."
    },

    {
      "role": "user", "content":"""The text is demarkated by triple backticks:```होरी एक गरीब किसान था, जो मुश्किल से गुजारा कर पा रहा था। अपनी कठिनाइयों के बावजूद, वह एक गाय खरीदने का सपना देखता था, जो गाँव में समृद्धि का प्रतीक थी।```.
       The theme is demarkated by double backticks:``होरी गाय क्यों खरीदना चाहता था?``.""",
    },

    {"role":"assistant","content": "Yes, this question is relevant to this text."
    },
    {
      "role":"user","content": f'''The text is demarkated by triple backticks:```{segment}```. 
                                   The question is demarkated by double backticks:``{question}``.''',
    }
  ] 

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  res = decoded[0]
  
  last_inst_index = res.rfind('[/INST]')
  if last_inst_index != -1:
      res = res[last_inst_index + len('[/INST]'):]
    
    
  # question_mark_index = res.find('?')
  # if question_mark_index != -1:
  #     res = res[:question_mark_index + 1]  
  
  if 'coherent' in res.strip():
    resfin.append(res)
    break
print(resfin)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[' No, this question is incoherent with the given text. The text pertains to a website selling a book about "Godan" by Premchand, which is not related to the topic of Hiera drowning in debt.</s>']


In [13]:
resfin

[]

In [ ]:
# Load the NLLB model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer_ts = AutoTokenizer.from_pretrained(model_name)
model_ts = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:

# Text to translate
for res in resfin:
    text = res

    # Tokenize the input text for the source language (English)
    tokenizer.src_lang = "eng_Latn"
    inputs = tokenizer_ts(text, return_tensors="pt")

    # Set the target language (Sanskrit) by using the corresponding token ID
    target_lang = "san_Deva"
    forced_bos_token_id = tokenizer_ts.convert_tokens_to_ids(target_lang)

    # Generate the translation with a limit on the number of new tokens generated
    translated_tokens = model_ts.generate(**inputs, forced_bos_token_id=forced_bos_token_id, max_new_tokens=50)

    # Decode the translated tokens to get the final translated text
    translated_text = tokenizer_ts.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # Print the translated text
    print(translated_text)
    print("\n")
